<a href="https://colab.research.google.com/github/JyothsnaThangudu/Projects/blob/main/WhatsApp_chat_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Packages


In [3]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from PIL import Image

from collections import Counter
from wordcloud import STOPWORDS, ImageColorGenerator, wordcloud

Import Dataset

In [4]:
conversation = 'WhatsApp Chat with Ramya.txt'

Find Time and Authors

In [14]:
def date_time(s):
  pattern = r'^\d{2}/\d{2}/\d{4}, \d{2}:\d{2} (AM|PM|am|pm) - '
  result = re.match(pattern,s)
  if result:
    return True
  return False

In [15]:
def find_author(s):
  s = s.split(":")
  if len(s) == 2:
    return True
  else:
    return False

In [16]:
def messages(line):
  splitline = line.split(' - ')
  dateTime = splitline[0]
  date,time = dateTime.split(',')
  message = " ".join(splitline[1:])

  if find_author(message):
    splitmessage = message.split(":")
    author = splitmessage[0]
    message = " ".join(splitmessage[1:])
  else:
    author = None

  return date,time,author,message



Prepare the Dataset

In [17]:
data = []
conversation = 'WhatsApp Chat with Ramya.txt'

with open(conversation,encoding="utf-8") as fp:
  fp.readline()
  messageBuffer = []
  date,time,author = None,None,None

  while True:
    line = fp.readline()
    if not line:
      break
    line = line.strip()
    if date_time(line):
      if len(messageBuffer) > 0:
        date.append([date,time,author,' '.join(messageBuffer)])
      messageBuffer.clear()
      date,time,author,message = messages(line)
      messageBuffer.append(message)
    else:
      messageBuffer.append(line)





In [19]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [20]:
df = pd.DataFrame(data,columns = ['Date','Time','Author','Message'])
df['Date'] = pd.to_datetime(df['Date'])
data = df.dropna()

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiments = SentimentIntensityAnalyzer()

data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

print(data.head(10))

Empty DataFrame
Columns: [Date, Time, Author, Message, Positive, Negative, Neutral]
Index: []


Validate the Sentiment

In [21]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def score(a,b,c):
  if a>b and a>c:
    print("POSITIVE")
  elif b>a and b>c:
     print("NEGATIVE")
  else:
    print("NEUTRAL")
score(x,y,z)

NEUTRAL
